In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import numpy as np 
import os 

import pandas as pd 
from sklearn.model_selection import train_test_split,  cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, linear_model
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, make_scorer, recall_score
from sklearn.svm import SVC

import warnings
from time import time
from scipy.stats import randint as sp_randint
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats.kde import gaussian_kde
from numpy import linspace,hstack
from pylab import plot,show,hist

Приготовимся к описанию нашего дата-сета; для начала выведем несколько первых строк с целью убедиться, что набор данных считался и готов к работе.

In [2]:
DS = pd.read_csv("fraud.csv", delimiter=',')  

Выведем первые несколько строк таблицы с набором наших данных:

In [3]:
DS.head()

,loanKey,rep_loan_date,first_loan,dpd_5_cnt,dpd_15_cnt,dpd_30_cnt,first_overdue_date,close_loans_cnt,federal_district_nm,TraderKey,...,payment_type_2,payment_type_3,payment_type_4,payment_type_5,past_billings_cnt,score_1,score_2,age,gender,bad_flag
0,307a838f523ff4963ff3d9436b04be90,2016-03-26,2015-09-28,1.0,0.0,0.0,2016-01-30,0.0,region_6,1679091c5a880faf6fb5e6087eb1b2dc,...,6,0,0,0,5.0,NaN,NaN,20.0,False,0
1,779ad1fcf8fa5a1fb863e725ba02f3ea,2016-10-30,2015-12-21,NaN,NaN,NaN,NaN,2.0,region_1,1679091c5a880faf6fb5e6087eb1b2dc,...,0,5,0,0,5.0,NaN,NaN,19.0,False,0
2,665d9bb66b1dcb65e89f922ac00f1f45,2016-06-29,2015-06-30,NaN,NaN,NaN,NaN,1.0,region_6,aab3238922bcc25a6f606eb525ffdc56,...,4,0,0,0,6.0,NaN,NaN,21.0,False,0
3,0af212b8130a0285ca67dfeb57c64a68,2015-12-01,2015-08-03,NaN,NaN,NaN,NaN,0.0,region_5,b6d767d2f8ed5d21a44b0e5886680cb9,...,0,0,0,0,3.0,NaN,NaN,33.0,False,0
4,5f13b34f42f732620491c282ae56bc23,2016-10-01,2015-08-30,NaN,NaN,NaN,NaN,2.0,region_3,a5771bce93e200c36f7cd9dfd0e5deaa,...,6,0,0,0,5.0,NaN,NaN,34.0,False,0


Количество полей данных (атрибутов):

In [4]:
DS.shape

(4156, 22)

Удалим те атрибуты, в которых более 5% отсутствующих значений. Это поля: past_billings_cnt, score_1, first_overdue_date, dpd_30_cnt, dpd_15_cnt, dpd_5_cnt, score_2.           
       
                

In [5]:
del DS['past_billings_cnt']
del DS['score_1']
del DS['first_overdue_date']
del DS['dpd_30_cnt']
del DS['dpd_15_cnt']
del DS['dpd_5_cnt']
del DS['score_2']
DS

,loanKey,rep_loan_date,first_loan,close_loans_cnt,federal_district_nm,TraderKey,payment_type_0,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,age,gender,bad_flag
0,307a838f523ff4963ff3d9436b04be90,2016-03-26,2015-09-28,0.0,region_6,1679091c5a880faf6fb5e6087eb1b2dc,0,0,6,0,0,0,20.0,False,0
1,779ad1fcf8fa5a1fb863e725ba02f3ea,2016-10-30,2015-12-21,2.0,region_1,1679091c5a880faf6fb5e6087eb1b2dc,0,0,0,5,0,0,19.0,False,0
2,665d9bb66b1dcb65e89f922ac00f1f45,2016-06-29,2015-06-30,1.0,region_6,aab3238922bcc25a6f606eb525ffdc56,0,0,4,0,0,0,21.0,False,0
3,0af212b8130a0285ca67dfeb57c64a68,2015-12-01,2015-08-03,0.0,region_5,b6d767d2f8ed5d21a44b0e5886680cb9,0,3,0,0,0,0,33.0,False,0
4,5f13b34f42f732620491c282ae56bc23,2016-10-01,2015-08-30,2.0,region_3,a5771bce93e200c36f7cd9dfd0e5deaa,0,0,6,0,0,0,34.0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4151,cf43bb28e8c5ecf07d8bb32a243d9e39,2016-06-11,2016-05-17,0.0,region_5,8f14e45fceea167a5a36dedd4bea2543,0,0,1,0,0,0,26.0,False,0
4152,4a31ec1c6d5e4809c4677e873f92b243,2016-08-06,2016-03-11,1.0,region_2,c20ad4d76fe97759aa27a0c99bff6710,0,0,4,0,0,0,38.0,False,1
4153,4140d1e8424e12c890d439e5472e2deb,2016-09-05,2016-04-30,1.0,region_2,c20ad4d76fe97759aa27a0c99bff6710,0,0,4,0,0,0,21.0,False,0
4154,8b03ab03763644b74c9c01030209206b,2016-10-14,2016-05-20,0.0,region_6,8f14e45fceea167a5a36dedd4bea2543,0,0,4,0,0,0,35.0,False,0


С таким же успехом можно удалить другим способом ещё 4 категориальных атрибута, которые не влияют на распознавание "подозрительности" тразакции: это их ID-коды, регион, где совершался платёж, а также "битый" столбец с полом, где все значения указаны как "False".

In [6]:
DS = DS.drop(['first_loan','loanKey', 'rep_loan_date', 'TraderKey','federal_district_nm','gender'],axis = 1)
DS

,close_loans_cnt,payment_type_0,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,age,bad_flag
0,0.0,0,0,6,0,0,0,20.0,0
1,2.0,0,0,0,5,0,0,19.0,0
2,1.0,0,0,4,0,0,0,21.0,0
3,0.0,0,3,0,0,0,0,33.0,0
4,2.0,0,0,6,0,0,0,34.0,0
...,...,...,...,...,...,...,...,...,...
4151,0.0,0,0,1,0,0,0,26.0,0
4152,1.0,0,0,4,0,0,0,38.0,1
4153,1.0,0,0,4,0,0,0,21.0,0
4154,0.0,0,0,4,0,0,0,35.0,0


In [7]:
DS.shape

(4156, 9)

# 2. Разделение набора данных на обучающую и тестовую выборки. 

Машинное обучение по своему определнию представляет собой обучение выделению некоторых свойств выборки данных и их применение на новых данных. Поэтому, исходя из общепринятой практики оценки алгоритма в машинном обучении, на втором этапе работы над курсовой необходимо вручную разбить данные на два отдельных набора. Первый — обучающая выборка, на которой будут изучаться свойства данных, а второй — тестовая выборка, на которой выделенные свойства будут тестироваться.

В процессе разделения будет использован случайный метод, который к тому же работает по умолчанию. Прочие методы было решено не использовать в силу специфики задачи. 

Разделим набор данных на обучающую и тестовую выборки с помощью метода train_test_split, но сначала превратим дата-сет во фрейм данных и определим имена его столбцов:

In [11]:
columns = "payment_type_0 payment_type_1 payment_type_2 payment_type_3 payment_type_4 payment_type_5 age".split() 
# объявляем имена необходимых (несущих информационную нагрузку) столбцов для работы над выборками
x = pd.DataFrame(DS, columns = columns)
y = DS.bad_flag # определяем целевую перменную как "y"

Теперь мы можем использовать функцию train_test_split для разделения набора данных на обучающую и тестовую выборки.

Пусть параметр test_size = 0,2 внутри функции указывает процент данных, которые должны быть сохранены для тестирования. 
Обычно это соотношения около 80/20 или 70/30. Разбиение данных на две этих выборки выглядит следующим образом:

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [18]:
x_train.shape

(3324, 7)

In [19]:
y_train.shape

(3324,)

In [20]:
x_test.shape

(832, 7)

In [21]:
y_test.shape

(832,)

# 3. Обучение нескольких моделей для решения выбранной задачи.
## 3.1 Алгоритм наивного байесовского классификатора.

Первой моделью решения задачи выступит алгоритм наивного Байесовского классификатора. Он, к тому же, считается одним из самых простых алгоритмов классификации. При этом ему удаётся работать не хуже, а то и лучше некоторых более сложных алгоритмов.

Здесь мы собираемся использовать модель GaussianNB, которая доступна в библиотеке Sckit-learn. Создание нашей наивной байесовской модели будет возможным с использованием метода GaussianNB():

In [37]:
NBC = GaussianNB()
NBC_2 = NBC.fit(x, y)

Сделаем прогнозы на основе разделения данных на целевую переменную и прочие:

In [38]:
expected = y
predicted = NBC.predict(x)

Что касается получения точности и статистики, то теперь имеет смысл создать отчёт о классификации, который будет содержать различные статистические данные, необходимые для оценки модели. И перед переходом к анализу метрик необходимо будет ввести важную концепцию для описания этих метрик в терминах ошибок классификации, а именно — confusion matrix (матрицу ошибок), которая имеет следующий вид:

![вид матрицы ошибок](https://sun9-24.userapi.com/BUpDzkAT__m1qptu5UFb6C60IAGpDND8ACsrfg/phRujTtXcwE.jpg "вид матрицы ошибок")

Здесь $ŷ$ — это ответ алгоритма на объекте, а $y$ — истинная метка класса на этом объекте.
Таким образом, ошибки классификации бывают двух видов: False Negative (FN) и False Positive (FP).

#### Матрица ошибок:

In [39]:
print(metrics.confusion_matrix(expected, predicted))

[[3702   37]
 [ 413    4]]


Интуитивно понятной, очевидной (и почти неиспользуемой) метрикой является $accuracy$ — доля правильных ответов алгоритма.
Для оценки качества работы алгоритма также введём метрики $precision$ (точность) и $recall$ (полнота). $Precision$ можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными, а $recall$ показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.

#### Классификационный отчет:

In [40]:
print(metrics.classification_report(expected, predicted))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      3739
           1       0.10      0.01      0.02       417

    accuracy                           0.89      4156
   macro avg       0.50      0.50      0.48      4156
weighted avg       0.82      0.89      0.85      4156



Оценка точности по тренировочной выборке:

In [47]:
display(NBC.score(x_train, y_train)) 

0.8895908543922985

Оценка точности по тестовой выборке:

In [48]:
display(NBC.score(x_test, y_test))

0.9002403846153846

## 3.2 Алгоритм итеративного дихотомизатора (ID3) (05-6/11)

Алгоритм итеративного дихотомизатора является одним из самых популярных алгоритмов, подразумевающих собой построение дерева решений. В данных методах сначала строится модель решений, которые принимаются на основе фактических значений атрибутов в данных. Решения распадаются на древовидные структуры до тех пор, пока не будет принято прогнозирующее решение для данной записи. Деревья решений обучаются на данных для задач классификации и регрессии. Деревья решений часто бывают быстрыми и точными и пользуются большой популярностью в машинном обучении.

In [49]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
ID3 = DecisionTreeClassifier()
ID3_fit = ID3.fit(x_train, y_train)

def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0] # true_negative
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1] # false_positive
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0] # false_negative
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1] # true_positive

scoring = {
    'tp' : make_scorer(tp), 
    'tn' : make_scorer(tn), 
    'fp' : make_scorer(fp), 
    'fn' : make_scorer(fn), 
    'accuracy' : make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'f1_score' : make_scorer(f1_score),
    'recall'   : make_scorer(recall_score)
}

sc = cross_validate(ID3_fit, x_train, y_train, cv = 5, scoring = scoring)

In [50]:
expected_1 = y
predicted_1 = ID3_fit.predict(x)

#### Матрица ошибок:

In [52]:
print(metrics.confusion_matrix(expected_1, predicted_1))

[[3706   33]
 [ 310  107]]


#### Классификационный отчет:

In [53]:
print(metrics.classification_report(expected_1, predicted_1))

              precision    recall  f1-score   support

           0       0.92      0.99      0.96      3739
           1       0.76      0.26      0.38       417

    accuracy                           0.92      4156
   macro avg       0.84      0.62      0.67      4156
weighted avg       0.91      0.92      0.90      4156



Оценка точности по обучающей выборке:

In [58]:
display(ID3.score(x_train, y_train))

0.9262936221419976

Оценка точности по тестовой выборке:

In [59]:
display(ID3.score(x_test, y_test))

0.8822115384615384

# 4. Выбор наиболее перспективной модели для решения поставленной задачи. Поиск оптимальных гиперпараметров (06/11).

In [53]:
# Не совсем понимаю, какая из моделей (наивный байес или ID3) является более перспективной. 
# судя по оценке точности тестовой выборки, которая более важна, как Вы сказали, перспективнее воспользоваться Байесом.
# но Вы также говорили, что по нему плохой результат получился. поэтому я в тупике

Один из способов настройки гиперпараметров состоит в том, чтобы заставить компьютер попробовать все возможные комбинации значений параметров. Для этого используем модуль GridSearchCV из библиотеки Scikit Learn. Попытаемся найти наилучшую комбинацию гиперпараметров (C, Gamma и Kernel) для построения классификатора для нашего набора данных.

In [ ]:
# Какие гиперпараметры Вы бы посоветовали мне поискать? (1)

# И правильно ли я понимаю, что нужно перед их поиском описывать каждый из них и причину, по которой мы именно его ищем?

# Какую выборку мне нужно использовать для работы с методами поиска гиперпараметров? (3)

### Метод GridSearch

In [107]:
warnings.filterwarnings("ignore")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
tuned_parameters = [{'kernel': ['rbf','sigmoid', 'linear'], 'gamma': [1e-3, 1e-4, 1e-2], 'C': [1, 10, 100, 1000]}] # (1)
scores = ['precision'] 

clf = GridSearchCV(SVC(), tuned_parameters, cv=5, scoring='%s_macro' % scores[0])
clf.fit(x_train, y_train) # (3)

print("Оптимальные гиперпараметры:")
print(clf.best_params_)

print("Grid-метрики:")
means = clf.cv_results_['mean_test_score']
for mean,params in zip(means, clf.cv_results_['params']):
    print("%0.3f for %r" % (mean, params))

Оптимальные гиперпараметры:
{'C': 100, 'gamma': 0.0001, 'kernel': 'sigmoid'}
Grid-метрики:
0.449 for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.477 for {'C': 1, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.449 for {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
0.449 for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.449 for {'C': 1, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.449 for {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
0.449 for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.449 for {'C': 1, 'gamma': 0.01, 'kernel': 'sigmoid'}
0.449 for {'C': 1, 'gamma': 0.01, 'kernel': 'linear'}
0.449 for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.477 for {'C': 10, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.449 for {'C': 10, 'gamma': 0.001, 'kernel': 'linear'}
0.449 for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.449 for {'C': 10, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.449 for {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
0.449 for {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
0.449 for {'C': 10, 'gamma': 0.01

Приведенный выше код перебирает силу и пробует каждую возможную комбинацию этих трех гиперпараметров, а затем выводит наилучшую комбинацию.

Плюсы: Этот метод получит требуемые оптимальные гиперпараметры.

Минусы: Это исчерпывающая операция. Если диапазон или число гиперпараметров велики, возможности могут исчисляться миллионами, и на завершение потребуется довольно много времени. 

Также можем опробовать случайный поиск. Как следует из названия, он будет пытаться случайным образом выбрать заранее определенное количество комбинаций гиперпараметров, которые дадут некоторое представление о том, какие из них будут оптимальными.

### Метод случайного поиска

In [60]:
rfc = RandomForestClassifier(n_estimators = 20)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

hp = {"max_depth": [4, 3, 2],
              "min_samples_split": sp_randint(2, 11),
              "criterion": ["gini", "entropy"],
              "bootstrap": [True, False]
              }

n_iter_search = 10
random_search = RandomizedSearchCV(rfc, param_distributions = hp,
                                   n_iter = n_iter_search, cv = 5, iid = False)

start = time()
random_search.fit(x_test, y_test) # (аналогично) какую выборку мне нужно здесь нужно использовать?
print("Метод случайного поиска занял %.2f секунд для нахождения %d"
      " следующих гиперпараметров:" % ((time() - start), n_iter_search))
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_search.cv_results_['params']):
  print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Метод случайного поиска занял 1.32 секунд для нахождения 10 следующих гиперпараметров:
0.907 (+/-0.005) for {'bootstrap': False, 'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 4}
0.909 (+/-0.004) for {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_split': 8}
0.909 (+/-0.004) for {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 8}
0.909 (+/-0.004) for {'bootstrap': False, 'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 9}
0.907 (+/-0.005) for {'bootstrap': False, 'criterion': 'gini', 'max_depth': 4, 'min_samples_split': 5}
0.907 (+/-0.005) for {'bootstrap': False, 'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 6}
0.907 (+/-0.005) for {'bootstrap': False, 'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 8}
0.909 (+/-0.004) for {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_split': 7}
0.909 (+/-0.004) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 2, 'mi

Плюсы: Гораздо быстрее, чем метод Grid Search.

Минусы: Может не возвращать оптимальную комбинацию гиперпараметров, которая вернула бы лучшую точность. Он также не учитывает прошлые оценки и продолжает итерации вне зависимости от результатов.